In [1]:
# Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Boolean
from sqlalchemy.orm import sessionmaker, relationship

In [2]:
# Create engine
engine = create_engine("sqlite:///birds.sqlite")

In [3]:
# Declaring a base object to model the table as an ORM class
Base = declarative_base()

In [4]:
# Define ORM class for `speciesList` table
class Species(Base):
    
    __tablename__ = 'species'

    id = Column(Integer, primary_key=True)
    Scientific_Name = Column(Text)
    Common_Name = Column(Text)
    Common_Name2 = Column(Text)
    Species_Code = Column(Text)
    Order = Column(Text)
    Audio_URL = Column(Text)
    Img_URL = Column(Text)
    Info_URL = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.ScientificName}"

In [5]:
# Define ORM class for `states_centroid` table
class StateCentroids(Base):
    
    __tablename__ = 'state_centroids'

    id = Column(Integer, primary_key=True)
    state_abbr = Column(Text)
    state = Column(Text)
    center_lat = Column(Float)
    center_lng = Column(Float)
    zoom = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.state_abbr}"

In [6]:
# Define ORM class for `states_centroid` table
class Regions(Base):
    
    __tablename__ = 'region_centroids'

    id = Column(Integer, primary_key=True)
    Region = Column(Text)
    Lat = Column(Float)
    Lng = Column(Float)
    Zoom = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.Region}"

In [7]:
# !rm birds.sqlite

In [8]:
# Creating the table with `create_all`
Base.metadata.create_all(engine)

In [9]:
# Verifying table names in the database
engine.table_names()

['region_centroids', 'species', 'state_centroids']

In [10]:
# Bulk insert each file into their respective table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile, encoding = "utf8")
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Regions.__table__, 'data/region_centroids.csv')
populate_table(engine, Species.__table__, 'data/scraped_us.csv')
populate_table(engine, StateCentroids.__table__, 'data/state_centroids.csv')

In [11]:
# Basic query to ensure data was entered correctly into the table `regions`
engine.execute("SELECT * FROM region_centroids LIMIT 5").fetchall()

[(1, 'Pacific Northwest ', 45.749471, -120.614503, 4),
 (2, 'Northern Plains ', 44.977582, -107.958253, 4),
 (3, 'Pacific Southwest ', 38.935396000000004, -120.02246299999999, 4),
 (4, 'Central Plains ', 38.935395, -98.555178, 4),
 (5, 'Southern Rockies ', 38.935395, -98.555178, 4)]

In [12]:
# Basic query to ensure data was entered correctly into the table `species`
engine.execute("SELECT * FROM species LIMIT 5").fetchall()

[(1, 'Numenius madagascariensis', 'Far Eastern curlew', 'Far Eastern curlew', 'faecur', 'Charadriiformes', 'no audio', 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/Far_Eastern_Curlew_cairns_RWD2.jpg/220px-Far_Eastern_Curlew_cairns_RWD2.jpg', 'https://en.wikipedia.org/wiki/Far_Eastern_curlew'),
 (2, 'Charadrius dubius', 'Little ringed plover', 'Little ringed plover', 'lirplo', 'Charadriiformes', 'no audio', 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Little_ringed_plover_%28Charadrius_dubius%29_Photograph_by_Shantanu_Kuveskar.jpg/220px-Little_ringed_plover_%28Charadrius_dubius%29_Photograph_by_Shantanu_Kuveskar.jpg', 'https://en.wikipedia.org/wiki/Little_ringed_plover'),
 (3, 'Motacilla tschutschensis', 'Eastern yellow wagtail', 'Eastern yellow wagtail', 'eaywag', 'Passeriformes', 'no audio', 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Motacilla_tschutschensis%2C_Tomohon%2C_North_Sulawesi.jpg/220px-Motacilla_tschutschensis%2C_Tomohon%2C_North_

In [13]:
# Basic query to ensure data was entered correctly into the table `species`
engine.execute("SELECT * FROM state_centroids LIMIT 5").fetchall()

[(1, 'AL', 'Alabama', 32.7794, -86.8287, 5),
 (2, 'AK', 'Alaska', 64.0685, -152.2782, 6),
 (3, 'AZ', 'Arizona', 34.2744, -111.6602, 5),
 (4, 'AR', 'Arkansas', 34.8938, -92.4426, 5),
 (5, 'CA', 'California', 37.1841, -119.4696, 5)]